###### Import requests and pandas libraries


In [1]:
import requests
import pandas as pd

###### Get the HTML of the Wiki page, convert into a table with help of read_html (read HTML tables into a list of DataFrame objects), remove cells with a borough that is Not assigned and if a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

In [2]:
wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipedia_page = requests.get(wiki)

df = pd.read_html(wikipedia_page.content, header=0)[0]
df2 = df[df.Borough != 'Not assigned']

for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']
        
df2.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


###### More than one neighborhood can exist in one postal code area. These two rows will be combined into one row with the neighborhoods separated with a comma as shown.

In [3]:
df3 = df2.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(lambda x:', '.join(x)).to_frame().reset_index()


In [4]:
df3

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


###### Use the .shape method to print the number of rows and columns of your dataframe.

In [5]:
df3.shape

(103, 3)

###### We need to get the latitude and the longitude coordinates of each neighborhood.

In [6]:
import io

url="http://cocl.us/Geospatial_data"
s=requests.get(url).content
df_geo=pd.read_csv(io.StringIO(s.decode('utf-8')))

###### Check the shape

In [7]:
df_geo.shape

(103, 3)

###### Merge dataframes on Postal Code. Reorder column names.

In [8]:
df_geo.columns = ['Postal Code', 'Latitude', 'Longitude']
df3 = pd.merge(df_geo, df3, on='Postal Code')


df3 = df3[['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
df3

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
